<a href="https://colab.research.google.com/github/hrbolek/learning/blob/master/notebooks/chemistry/introduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip -q install rdkit py3Dmol

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.2/36.2 MB 55.6 MB/s eta 0:00:00


In [3]:
from rdkit import Chem
from rdkit.Chem import AllChem, Descriptors
import py3Dmol

mol = Chem.AddHs(Chem.MolFromSmiles("c1ccccc1O"))  # fenol
AllChem.EmbedMolecule(mol, AllChem.ETKDG())
AllChem.UFFOptimizeMolecule(mol)

mw = Descriptors.MolWt(mol)
print("Mol. hmotnost:", round(mw, 2))

# 3D náhled v Colabu
mb = Chem.MolToMolBlock(mol)
view = py3Dmol.view(width=400, height=300)
view.addModel(mb, 'sdf'); view.setStyle({'stick':{}})
view.zoomTo(); view.show()

Mol. hmotnost: 94.11


3Dmol.js failed to load for some reason. Please check your browser console for error messages.